In [25]:
import numpy as np;
import sys;
import os;
import glob;
import random;
import wavio;
import copy;
from random import shuffle;
# import librosa.display
# import matplotlib.pyplot as plt
import wave
from audiomentations import AddGaussianNoise
from audiomentations import AddBackgroundNoise, PolarityInversion
import array

In [2]:
import soundfile as sf

In [33]:
sys.path.append("../../../Libs/")
sys.path.append("../../../src/SharedLibs/")

In [36]:
from SharedLibs import getFileList
from sys_untity import ConvertSR;

In [15]:
from IPython.display import Audio;
import argparse

## Shared Codes

In [37]:
def save_wavefile(output_path=None, sig=None):
    output_file = wave.Wave_write(output_path)
    output_file.setnchannels(1);
    output_file.setframerate(20000);
    output_file.setsampwidth(2) #2bytes per sample.
    # output_file.setparams(params) #nchannels, sampwidth, framerate, nframes, comptype, compname
    output_file.writeframes(array.array('h', sig.astype(np.int16)).tobytes(), )
    output_file.close()

### Shared Data

In [6]:
sampling_rate = 20000;
desired_wav_len = 30225;#int(np.ceil(1.5 * sampling_rate));
clear_clip_file = "../../../datasets/CurrentUse/mixed_sounds/clear_clips/clear_clip.wav" 
one_half_sec_wav_name_fmt = "{}_{}.wav";
src_eng_help_dir = "../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s1_processed_clear_clips/";
head_clip_save_dir = "../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_processed_1.5sec/head/";
middle_clip_save_dir = "../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_processed_1.5sec/middle/";
tail_clip_save_dir = "../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_processed_1.5sec/tail/";
clear_eng_help_wav_list = getFileList(src_eng_help_dir);
clear_clip = wavio.read(clear_clip_file).data.T[0];
len_of_clear_clip = len(clear_clip);

In [7]:
print(f"count of clear english help wav files:{len(clear_eng_help_wav_list)}");
# print(os.path.basename(raw_eng_help_wav_list[0])[:-4])
# print(len(clear_clip))
# test_data = wavio.read(raw_eng_help_wav_list[0]).data.T[0];
# print(f"len of test_data:{len(test_data)}, shape:{test_data.shape}");
# res_data = np.append(test_data, clear_clip);
# print(f"len of res_data:{len(res_data)}, shape:{res_data.shape}");
# 
# print(len([i for i in clear_clip]))
# for i in raw_eng_help_wav_list:
#     print(i)

count of clear english help wav files:101


## Data Visualization Codes

In [12]:
# import librosa.display
# import matplotlib.pyplot as plt

In [11]:
# def plot_signal_and_augmented_signal(signal, augmented_signal, sr):
#     fig, ax = plt.subplots(nrows=2)
#     librosa.display.waveplot(signal, sr=sr, ax=ax[0])
#     ax[0].set(title="Original signal")
#     librosa.display.waveplot(augmented_signal, sr=sr, ax=ax[1])
#     ax[1].set(title="Augmented signal")
#     plt.show()

## putting speech signal in the head and extending clip to 1.5 Sec

In [9]:
def extend_at_head_main(suffix_word="head", upperlel_save_dir=None):
    counter = 0;
    for w in clear_eng_help_wav_list:
        tmp_data = wavio.read(w).data.T[0];
        len_data = len(tmp_data);
        result = None;
        file_name = one_half_sec_wav_name_fmt.format(os.path.basename(w)[:-4],suffix_word);
        save_full_name = os.path.join(upperlel_save_dir, file_name)
        # print(save_full_name)
        if len_data < desired_wav_len:
            result = np.append(tmp_data, clear_clip[:desired_wav_len-len_data]);
        elif len_data > desired_wav_len:
            result = tmp_data[:desired_wav_len];
        else:
            result = tmp_data;
        sf.write(save_full_name, result, sampling_rate);
        counter += 1;
        print(f"{counter}:writing wav file to {save_full_name}");

In [10]:
# extend_at_head_main(upperlel_save_dir=head_clip_save_dir);

## putting speech signal in the middle and extending clip to 1.5 Sec

In [11]:
def insert_middle_logic(wanted_len=None, src_wav_len=None):
    dif_len_pos = int(wanted_len - src_wav_len);
    half_dif_len = int(np.ceil(dif_len_pos/2));
    start_pos = half_dif_len;
    end_pos = start_pos + src_wav_len;
    return start_pos, end_pos

In [12]:
def extend_at_middle_main(suffix_word="middle",upperlel_save_dir=None):
    counter = 0;
    for w in clear_eng_help_wav_list:
        tmp_data = wavio.read(w).data.T[0];
        len_data = len(tmp_data);
        result = None;
        file_name = one_half_sec_wav_name_fmt.format(os.path.basename(w)[:-4],suffix_word);
        save_full_name = os.path.join(upperlel_save_dir, file_name);
        # print(save_full_name)
        if len_data < desired_wav_len:
            s_pos, e_pos = insert_middle_logic(wanted_len=desired_wav_len, src_wav_len=len_data);
            result = clear_clip[:s_pos].copy();
            result = np.append(result, tmp_data[:e_pos])
            if e_pos < desired_wav_len:
                result = np.append(result, clear_clip[:desired_wav_len-e_pos]);
        elif len_data > desired_wav_len:
            result = tmp_data[:desired_wav_len];
        else:
            result = tmp_data;
        sf.write(save_full_name, result, sampling_rate);
        counter += 1;
        print(f"{counter}:writing wav file to {save_full_name}");

In [13]:
# extend_at_middle_main(upperlel_save_dir=middle_clip_save_dir)

## putting speech signal in the tail and extending clip to 1.5 Sec

In [14]:
def extend_at_tail_main(suffix_word="tail", upperlel_save_dir=None):
    counter = 0;
    for w in clear_eng_help_wav_list:
        tmp_data = wavio.read(w).data.T[0];
        len_data = len(tmp_data);
        result = None;
        file_name = one_half_sec_wav_name_fmt.format(os.path.basename(w)[:-4],suffix_word);
        save_full_name = os.path.join(upperlel_save_dir, file_name)
        # print(save_full_name)
        if len_data < desired_wav_len:
            wanted_clear_clip_len = desired_wav_len-len_data;
            result = clear_clip[:wanted_clear_clip_len].copy();
            result = np.append(result, tmp_data);
        elif len_data > desired_wav_len:
            result = tmp_data[:desired_wav_len];
        else:
            result = tmp_data;
        sf.write(save_full_name, result, sampling_rate);
        counter += 1;
        print(f"{counter}:writing wav file to {save_full_name}");

In [15]:
# extend_at_tail_main(upperlel_save_dir=tail_clip_save_dir)

## mix with noises

### mixed with noises shared data

In [8]:
home_chat_noises_20K_dir = "../../CurrentUse/mixed_sounds/home_chat_noises/20K/"
help_onehalfsec_clips_head_dir = "../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_processed_1.5sec/head/";
help_onehalfsec_clips_middle_dir = "../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_processed_1.5sec/middle/";
help_onehalfsec_clips_tail_dir = "../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_processed_1.5sec/tail/";
mixed_help_wav_fm = "{}_hcn.wav";
mixed_help_save_dir_head = "../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s4_processed_with_home_chat_noises/head/";
mixed_help_save_dir_middle = "../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s4_processed_with_home_chat_noises/middle/";
mixed_help_save_dir_tail = "../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s4_processed_with_home_chat_noises/tail/";
mixed_whitenoise_help_wav_fm = "{}_hwn_{}.wav";
mixed_whitenoise_help_save_dir_head = "../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s3_mixed_with_white_noises/head/";
mixed_whitenoise_help_save_dir_middle = "../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s3_mixed_with_white_noises/middle/";
mixed_whitenoise_help_save_dir_tail = "../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s3_mixed_with_white_noises/tail/";
# home_chat_noise_20K_fm = "{}_20K.wav";
home_chat_noises_list = getFileList(home_chat_noises_20K_dir)
# help_onehalfsec_clips_head_list = getFileList(help_onehalfsec_clips_head_dir);
# help_onehalfsec_clips_middle_list = getFileList(help_onehalfsec_clips_middle_dir);
# help_onehalfsec_clips_tail_list = getFileList(help_onehalfsec_clips_tail_dir);
# print(f"number of chat noises files:{len(home_chat_noises_list)}")

In [9]:
sound_list_head = getFileList(help_onehalfsec_clips_head_dir);
sound_list_middle = getFileList(help_onehalfsec_clips_middle_dir);
sound_list_tail = getFileList(help_onehalfsec_clips_tail_dir);

In [15]:
# print(os.path.basename(home_chat_noises_list[0])[:-10])
# for n in home_chat_noises_list:
#     fname = home_chat_noise_20K_fm.format(os.path.basename(n)[:-10])
#     save_path = os.path.join(home_chat_noises_20K_dir,fname);
#     ConvertSR(n,save_path,sampling_rate);
#     print(f"convert to {save_path}");

In [16]:
# for t in range(10):
#     print(random.choice(home_chat_noises_list))

In [18]:
def cal_adjusted_rms(clean_rms, snr):
    a = float(snr) / 20
    noise_rms = clean_rms / (10**a) 
    return noise_rms

def cal_amp(wf):
    buffer = wf.readframes(wf.getnframes())
    # The dtype depends on the value of pulse-code modulation. The int16 is set for 16-bit PCM.
    amptitude = (np.frombuffer(buffer, dtype="int16")).astype(np.float64)
    return amptitude

def cal_rms(amp):
    return np.sqrt(np.mean(np.square(amp), axis=-1))

def mix_with_noise(clean_wav_file=None, noise_wav_file=None, snr=30, save_path=None):
    clean_wav = wave.open(clean_file, "r");
    noise_wav = wave.open(noise_file, "r")

    clean_amp = cal_amp(clean_wav);
    noise_amp = cal_amp(noise_wav);

    clean_rms = cal_rms(clean_amp);

    start = random.randint(0, len(noise_amp)-len(clean_amp))
    divided_noise_amp = noise_amp[start: start + len(clean_amp)]
    noise_rms = cal_rms(divided_noise_amp)

    adjusted_noise_rms = cal_adjusted_rms(clean_rms, snr)

    adjusted_noise_amp = divided_noise_amp * (adjusted_noise_rms / noise_rms) 
    mixed_amp = (clean_amp + adjusted_noise_amp)

    #Avoid clipping noise
    max_int16 = np.iinfo(np.int16).max
    min_int16 = np.iinfo(np.int16).min
    if mixed_amp.max(axis=0) > max_int16 or mixed_amp.min(axis=0) < min_int16:
        if mixed_amp.max(axis=0) >= abs(mixed_amp.min(axis=0)): 
            reduction_rate = max_int16 / mixed_amp.max(axis=0)
        else :
            reduction_rate = min_int16 / mixed_amp.min(axis=0)
        mixed_amp = mixed_amp * (reduction_rate)
        clean_amp = clean_amp * (reduction_rate)
    
    return mixed_amp

    
"""
clean_wav = wave.open(clean_file, "r")
noise_wav = wave.open(noise_file, "r")

clean_amp = cal_amp(clean_wav)
noise_amp = cal_amp(noise_wav)

clean_rms = cal_rms(clean_amp)

start = random.randint(0, len(noise_amp)-len(clean_amp))
divided_noise_amp = noise_amp[start: start + len(clean_amp)]
noise_rms = cal_rms(divided_noise_amp)

snr = args.snr
adjusted_noise_rms = cal_adjusted_rms(clean_rms, snr)

adjusted_noise_amp = divided_noise_amp * (adjusted_noise_rms / noise_rms) 
mixed_amp = (clean_amp + adjusted_noise_amp)

#Avoid clipping noise
max_int16 = np.iinfo(np.int16).max
min_int16 = np.iinfo(np.int16).min
if mixed_amp.max(axis=0) > max_int16 or mixed_amp.min(axis=0) < min_int16:
    if mixed_amp.max(axis=0) >= abs(mixed_amp.min(axis=0)): 
        reduction_rate = max_int16 / mixed_amp.max(axis=0)
    else :
        reduction_rate = min_int16 / mixed_amp.min(axis=0)
    mixed_amp = mixed_amp * (reduction_rate)
    clean_amp = clean_amp * (reduction_rate)
"""
    


'\nclean_wav = wave.open(clean_file, "r")\nnoise_wav = wave.open(noise_file, "r")\n\nclean_amp = cal_amp(clean_wav)\nnoise_amp = cal_amp(noise_wav)\n\nclean_rms = cal_rms(clean_amp)\n\nstart = random.randint(0, len(noise_amp)-len(clean_amp))\ndivided_noise_amp = noise_amp[start: start + len(clean_amp)]\nnoise_rms = cal_rms(divided_noise_amp)\n\nsnr = args.snr\nadjusted_noise_rms = cal_adjusted_rms(clean_rms, snr)\n\nadjusted_noise_amp = divided_noise_amp * (adjusted_noise_rms / noise_rms) \nmixed_amp = (clean_amp + adjusted_noise_amp)\n\n#Avoid clipping noise\nmax_int16 = np.iinfo(np.int16).max\nmin_int16 = np.iinfo(np.int16).min\nif mixed_amp.max(axis=0) > max_int16 or mixed_amp.min(axis=0) < min_int16:\n    if mixed_amp.max(axis=0) >= abs(mixed_amp.min(axis=0)): \n        reduction_rate = max_int16 / mixed_amp.max(axis=0)\n    else :\n        reduction_rate = min_int16 / mixed_amp.min(axis=0)\n    mixed_amp = mixed_amp * (reduction_rate)\n    clean_amp = clean_amp * (reduction_rate)\

In [ ]:
def mixed_with_noises(src_wav_dir=None, noise_wav_dir=None, save_wav_dir=None, suffix_word=None):
    counter = 0;
    for f in sound_list_head:
        transform = AddGaussianNoise(min_amplitude=0.0001,max_amplitude=0.0015,p=1.0);
        # mixed_ratio = round(random.uniform(0.001,0.015),3);#get mixed ratio between 0.05~0.1
        tmp_wav = wavio.read(f).data.T[0];
        file_name = mixed_whitenoise_help_wav_fm.format(os.path.basename(f)[:-9],suffix_word);
        save_full_path = os.path.join(save_wav_dir, file_name);
        # mixed_wav = add_white_noise(tmp_wav, mixed_ratio);
        mixed_wav = transform(tmp_wav, sample_rate=sampling_rate)
        sf.write(save_full_path, mixed_wav, sampling_rate);
        counter += 1;
        print(f"{counter}: save to:{save_full_path}");

In [29]:
def save_waveform(output_path, sig):
    output_file = wave.Wave_write(output_path)
    output_file.setnchannels(1);
    output_file.setframerate(20000);
    output_file.setsampwidth(2) #2bytes per sample.
    # output_file.setparams(params) #nchannels, sampwidth, framerate, nframes, comptype, compname
    output_file.writeframes(array.array('h', sig.astype(np.int16)).tobytes(), )
    output_file.close()

In [30]:
test_f007_man_wavfile = "../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_processed_1.5sec/head/man_clear_clip_help_007_71_head.wav";
home_noise_1 = "../../CurrentUse/mixed_sounds/home_chat_noises_1.5sec/"
test_f007_man_wav = wavio.read(test_f007_man_wavfile).data.T[0];
bgtransform = AddBackgroundNoise(
    sounds_path="../../CurrentUse/mixed_sounds/home_chat_noises_1.5sec/CR_lounge_270110_1632.s600_chunk15_20K.wav",
    min_snr_in_db=3.0,
    max_snr_in_db=30.0,
    noise_transform=PolarityInversion(),
    p=1.0
)

mixedbg_sound = bgtransform(test_f007_man_wav, sample_rate=20000)
save_waveform("../../CurrentUse/tmp/mixbgsound1.wav", mixedbg_sound)
# sf.write("../../CurrentUse/tmp/mixbgsound1.wav", mixedbg_sound, samplerate=20000);

In [19]:
def random_gain(signal, min_factor=0.1, max_factor=0.12):
    gain_rate = random.uniform(min_factor, max_factor)
    augmented_signal = signal * gain_rate
    return augmented_signal

In [20]:
def time_stretch(signal, time_stretch_rate):
    """Time stretching implemented with librosa:
    https://librosa.org/doc/main/generated/librosa.effects.pitch_shift.html?highlight=pitch%20shift#librosa.effects.pitch_shift
    """
    return librosa.effects.time_stretch(signal, time_stretch_rate)

In [41]:
round(random.uniform(0.05,0.1),2)

0.07

In [52]:
# test_noise_wav = wavio.read(home_chat_noises_list[2]).data.T[0];
# print(home_chat_noises_list[2])
# Audio(test_noise_wav,rate=sampling_rate);

In [31]:
test_head_wav = wavio.read(sound_list_head[1]).data.T[0];
Audio(test_head_wav,rate=sampling_rate)

In [40]:
_noise_percent_factor = 0.1;
test_aug_wav = add_white_noise(test_head_wav,_noise_percent_factor);
Audio(test_aug_wav,rate=sampling_rate)

In [42]:
def add_white_noise(signal, noise_percentage_factor):
    noise = np.random.normal(0, signal.std(), signal.size)
    augmented_signal = signal + noise * noise_percentage_factor
    return augmented_signal

In [43]:
def mixed_with_white_noises(src_wav_dir=None, save_wav_dir=None, suffix_word=None):
    counter = 0;
    for f in sound_list_head:
        # transform = AddGaussianNoise(min_amplitude=0.0001,max_amplitude=0.0015,p=1.0);
        mixed_ratio = round(random.uniform(0.05,0.1),4);#get mixed ratio between 0.05~0.1
        tmp_wav = wavio.read(f).data.T[0];
        file_name = mixed_whitenoise_help_wav_fm.format(os.path.basename(f)[:-9],suffix_word);
        save_full_path = os.path.join(save_wav_dir, file_name);
        mixed_wav = add_white_noise(tmp_wav, mixed_ratio);
        # mixed_wav = transform(tmp_wav, sample_rate=sampling_rate)
        save_wavefile(save_full_path, mixed_wav);
        # sf.write(save_full_path, mixed_wav, sampling_rate);
        counter += 1;
        print(f"{counter}: save to:{save_full_path}");

In [47]:
# mixed_with_white_noises(src_wav_dir=help_onehalfsec_clips_tail_dir, save_wav_dir=mixed_whitenoise_help_save_dir_tail, suffix_word="tail")

In [18]:
# test_wav_11 =  wavio.read(sound_list_head[0]).data.T[0];
# print(test_wav_11)

In [48]:
peek_wav_file = "../../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s3_mixed_with_white_noises/head/man_clear_clip_help_011_71_hwn_head.wav"
peek_wav = wavio.read(peek_wav_file).data.T[0];
print(peek_wav)

[ 258 -218  -75 ... -175  169  395]
